In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain, combinations
from scipy.stats import wilcoxon

In [2]:
baseline_models = pd.read_csv("baseline_models.csv")

In [3]:
first_val = baseline_models["Bitcoin"].reset_index(drop=True).values[0]
last_val = baseline_models["Bitcoin"].reset_index(drop=True).values[-1]

In [4]:
all_trades = pd.read_csv("final_scores_trading.csv")

In [5]:
all_options = [
    'LSTM_1_strategy1_dollars',
    'LSTM_1_strategy2_dollars',
    'GRU_1_strategy1_dollars',
    'GRU_1_strategy2_dollars',
    'LSTM_15_strategy1_dollars',
    'LSTM_15_strategy2_dollars',
    'GRU_15_strategy1_dollars',
    'GRU_15_strategy2_dollars'
               ]

In [6]:
#perform wilcoxen test for every combination of trading scores
all_tests = {x:{} for x in all_options}
for x,y in combinations(all_options,2):  
        
        strat_1_change = all_trades[x].rolling(1).sum().pct_change().values #change per min strat 1 
        strat_2_change = all_trades[y].rolling(1).sum().pct_change().values #change per min strat 2 
        difference = [] 
        for i in range(1,len(strat_2_change)):
            difference.append(strat_1_change[i]- strat_2_change[i]) #difference in change per minute 
        difference = difference[1:] #since we did rolling window first variable is nan, remove this one 
        stat,p = wilcoxon(difference) #testscores
        all_tests[x][y] = p #tore testsscores



In [7]:
#make a dataframe that contains all the tests scores

scores_tests = pd.DataFrame(columns=all_options,index=all_options)
for i in all_options:
    for j in all_options:
        if i == j:
            scores_tests[i].loc[j] = 'x'
        else:
            try: #don't know if the dictionary says [strat_1][strat_2] or other way around. So in try except 
                p_val = round(all_tests[i][j],4)
                if p_val <=0.05:
                    p_val =str(p_val) + "*"
                scores_tests[i].loc[j] = p_val
                scores_tests[j].loc[i] = p_val
            except:
                p_val = round(all_tests[j][i],4)
                if p_val <=0.05:
                    p_val =str(p_val) + "*"
                scores_tests[i].loc[j] = p_val
                scores_tests[j].loc[i] = p_val
                
                
                

In [8]:
scores_tests.to_csv("outcomes_wilcoxon.csv") #store

In [9]:
scores_tests.head()

,LSTM_1_strategy1_dollars,LSTM_1_strategy2_dollars,GRU_1_strategy1_dollars,GRU_1_strategy2_dollars,LSTM_15_strategy1_dollars,LSTM_15_strategy2_dollars,GRU_15_strategy1_dollars,GRU_15_strategy2_dollars
LSTM_1_strategy1_dollars,x,0.7985,0.0237*,0.0223*,0.6421,0.579,0.2716,0.7236
LSTM_1_strategy2_dollars,0.7985,x,0.0221*,0.0174*,0.6084,0.332,0.0668,0.3619
GRU_1_strategy1_dollars,0.0237*,0.0221*,x,0.1359,0.0119*,0.0067*,0.002*,0.007*
GRU_1_strategy2_dollars,0.0223*,0.0174*,0.1359,x,0.0139*,0.0058*,0.0016*,0.0069*
LSTM_15_strategy1_dollars,0.6421,0.6084,0.0119*,0.0139*,x,0.5184,0.5528,0.7991
